In [2]:
import lyricsgenius

genius = lyricsgenius.Genius('9dKJP0ay75Pm4xmOvUUCzejH-ruwxF4fhmy90B9QnNzKCMuiL6GaFNFo87P26twa')


In [3]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
import argparse
import logging
import spotipy
import pandas as pd
import numpy as np
import requests
from spotipy import oauth2
import re

import credentials

In [4]:
# Gain access to mainupation
SCOPE = ('user-read-recently-played,user-library-read,user-read-currently-playing,playlist-read-private,playlist-modify-private,playlist-modify-public,user-read-email,user-modify-playback-state,user-read-private,user-read-playback-state')
sp_oauth = oauth2.SpotifyOAuth(credentials.SPOTIPY_CLIENT_ID,credentials.SPOTIPY_CLIENT_SECRET, credentials.SPOTIPY_REDIRECT_URI ,scope=SCOPE )

#click "Accept" in your browser when the auth window pops up
code = sp_oauth.get_auth_response(open_browser=True)
token = sp_oauth.get_access_token(code)
refresh_token = token['refresh_token']
sp = spotipy.Spotify(auth=token['access_token'])
username = sp.current_user()['id']

/var/folders/pz/lc5f5cgd1m5_ghmjddlz8r600000gn/T/ipykernel_22145/2781803620.py:7: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token = sp_oauth.get_access_token(code)


In [40]:
track_info = sp.track('6rEc69BeR8xHy3kN1ZFCsX')

# Extract the track name and artist name from the track info
track_name = track_info['name']
artist_name = track_info['artists'][0]['name']

In [42]:
song = genius.search_song(track_name, artist=artist_name)
lyrics = song.lyrics

Searching for "Selfish" by Tensnake...
Done.


In [43]:
print(lyrics)

Selfish LyricsThere was a time when it was all about us, nobody more
Seems like every other night you're out that door
And I don't know what you're doing, maybe pursuing another man
There was a time when I'd lay it all down on the line
And bare everything I had til you were mine
But now all bets are off, can't afford the cost of loving you

Why you so selfish? Be by my side
I'll runaway and hide til I can't help it
I know you're not here with me
Just how much more giving can I take?
There was a time when you didn't care at all if you were poor
Now all you seem to want is to have more
And the cash you keep on spending
It could be mending our broken lives
Back in the day, before all you cared about was yourself
Your concerns were more for love and heart and health
But I won't stick a brighter day
And the only cure for me is your love

Why you so selfish?
Be by my side, I'll runaway and hide til I can't help it
I know you're not here with me
Just how much more giving can I take?

Take it,

# Adding in NLP Now

In [44]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer('english')
stop_words = set(stopwords.words('english'))

def preprocess_lyrics(lyrics):
    # Tokenize the lyrics into sentences
    sentences = sent_tokenize(lyrics)
    
    # Tokenize each sentence into words
    words = [word_tokenize(sentence) for sentence in sentences]
    
    # Remove stop words and stem the remaining words
    stemmed_words = []
    for sentence in words:
        for word in sentence:
            if word.lower() not in stop_words:
                stemmed_word = stemmer.stem(word)
                stemmed_words.append(stemmed_word)
    
    # Combine the stemmed words into a single string
    preprocessed_lyrics = ' '.join(stemmed_words)
    
    return preprocessed_lyrics

In [61]:
import svgling
import nltk
tokens = nltk.word_tokenize(lyrics)
print(tokens)

['Selfish', 'LyricsThere', 'was', 'a', 'time', 'when', 'it', 'was', 'all', 'about', 'us', ',', 'nobody', 'more', 'Seems', 'like', 'every', 'other', 'night', 'you', "'re", 'out', 'that', 'door', 'And', 'I', 'do', "n't", 'know', 'what', 'you', "'re", 'doing', ',', 'maybe', 'pursuing', 'another', 'man', 'There', 'was', 'a', 'time', 'when', 'I', "'d", 'lay', 'it', 'all', 'down', 'on', 'the', 'line', 'And', 'bare', 'everything', 'I', 'had', 'til', 'you', 'were', 'mine', 'But', 'now', 'all', 'bets', 'are', 'off', ',', 'ca', "n't", 'afford', 'the', 'cost', 'of', 'loving', 'you', 'Why', 'you', 'so', 'selfish', '?', 'Be', 'by', 'my', 'side', 'I', "'ll", 'runaway', 'and', 'hide', 'til', 'I', 'ca', "n't", 'help', 'it', 'I', 'know', 'you', "'re", 'not', 'here', 'with', 'me', 'Just', 'how', 'much', 'more', 'giving', 'can', 'I', 'take', '?', 'There', 'was', 'a', 'time', 'when', 'you', 'did', "n't", 'care', 'at', 'all', 'if', 'you', 'were', 'poor', 'Now', 'all', 'you', 'seem', 'to', 'want', 'is', 'to

In [62]:
# remove stop words
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

# Remove stop words from each string
for i in range(len(tokens)):
    
    # Remove stop words from the list of words
    filtered_words = [word for word in tokens if word.casefold() not in stop_words]
    
    # Join the filtered words back into a string and replace the original string
    tokens[i] = ' '.join(filtered_words)

    # Remove any resulting empty strings
    if tokens[i] == '':
        tokens.pop(i)

print(tokens)


KeyboardInterrupt: 

In [66]:
from textblob import TextBlob
textblob_lyrics = TextBlob(lyrics)
textblob_lyrics.sentiment

Sentiment(polarity=0.08719135802469136, subjectivity=0.5785493827160493)

In [57]:
from nltk.probability import FreqDist

# Combine the preprocessed lyrics from all the songs into a single string
all_lyrics = ' '.join(tokens)

# Tokenize the lyrics into individual words
all_words = word_tokenize(all_lyrics)

# Calculate the frequency distribution of the words
freq_dist = FreqDist(all_words)

# Get the 10 most common words
most_common_words = freq_dist.most_common(10)

print(most_common_words)

[(',', 8), ("n't", 6), ('?', 5), ('take', 5), ("'re", 4), ('time', 3), ('know', 3), ('til', 3), ('ca', 3), ('selfish', 3)]
